In [40]:
import heapq #El módulo heapq implementa colas de prioridad (heaps)

In [41]:
class Node:
    def __init__(self, position, parent=None, path_cost=0, action =None): #AGREGAR ACTION
        self.position = position # Cordenada x,y del robot en el laberinto 
        self.parent = parent # Nodo padre desde donde se llego a este nodo
        self.path_cost = path_cost # Costo acumulado del camino desde donde se empezo hasta este nodo
        self.action = action # Acción tomada desde el nodo padre hasta este nodo, ejemplo: Arriba, abajo, derecha, izquierda

    def __lt__(self, other):
        return self.path_cost < other.path_cost # Compara los dos nodos en la cola de prioridad, y expande el camino más corto primero

class Problem:
    def __init__(self, initial, goal, actions, maze):
        self.initial = initial  # Coordenada x,y del robot en el laberinto de inicio
        self.goal = goal # Coordenada x,y del robot en el laberinto de fin, el objetivo
        self.actions = actions # Lista de acciones posibles para el robot, ejemplo: Arriba, abajo, derecha, izquierda
        self.maze = maze # Representación gráfica del laberinto en una matriz
        
    #DEFINA la Class problem como lo considere necesario, puede basarse del ejemplo de Bucharest

In [42]:
def reconstruct_path(node):  #AJUSTAR FUNCIONES PARA ADEMAS DE LAS POSICIONES, MOSTRAR LAS ACCIONES TOMADAS
    path = [] 
    actions = [] 
    while node:  
        path.append(node.position)
        if node.action:
            actions.append(node.action)
        node = node.parent
    path.reverse()
    actions.reverse()
    return path, actions  

"""
Explicación del algoritmo:
Se inicia con el nodo final, que es aquel que le entra como parámetro (E). Luego, se recorre la cadena de nodos padres (parent), retrocediendo desde la mete hasta la posición inicial
(S).
Se almacenan en el path la lista de coordenadas dentro de la matriz y las acciones tomadas en cada paso.
Luego, se le hace reverse a la lista para que los pasos queden como si hubiern sido realizados de S hasta E y no al revés. 
"""


'\nExplicación del algoritmo:\nSe inicia con el nodo final, que es aquel que le entra como parámetro (E). Luego, se recorre la cadena de nodos padres (parent), retrocediendo desde la mete hasta la posición inicial\n(S).\nSe almacenan en el path la lista de coordenadas dentro de la matriz y las acciones tomadas en cada paso.\nLuego, se le hace reverse a la lista para que los pasos queden como si hubiern sido realizados de S hasta E y no al revés. \n'

In [43]:
def find_exit(maze):
    start = (1, 1)  # Posición inicial basado en la documentación suministrada
    end = (1, 6)    # Posición de la salida basado en la documentación suministrada

    #DEFINA el conjunto de actions posibles#
    actions = {
        "up":(-1,0),
        "down":(1, 0),
        "right":(0, 1),
        "left":(0, -1)
        
    }

    problem=Problem(start, end, actions, maze)#COMPLETE LA DEFINICIÓN DEL OBJETO Y ADAPTELO EN LOS PUNTOS QUE LO REQUIERAN

    def manhatan_distance(pos, goal):
        return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])  # Distancia de Manhattan
    """
    La distancia Manhattan refleja el número mínimo de movimientos requeridos en una cuadrícula donde solo puedes moverte hacia arriba, abajo, derecha o izquierda.
    Ejemplo práctico: Lo que se quiere conseguir con este maze problem
    Sí tengo un robot en la posición (1,1) y quiero llegar a la posición (1,6), la distancia es: 
    abs(1-1) + abs(6-1) = 5. Así, sé que en el mejor de los casos el robot necesitará al menos 5 pasos para llegar al destino. 
    """

    def get_neighbors(pos):  #ESTA ES LA FUNCIÓN QUE DEBERIA AJUSTAR PARA HACER TRACKING DE LOS MOVIMIENTOS (Up, Down, Right, Left)
        neighbors = [] #lista de vecinos
        for move in problem.actions.keys():
            desplazamiento = problem.actions[move]
            neighbor = (pos[0] + desplazamiento[0], pos[1] + desplazamiento[1])

            if maze[neighbor[0]][neighbor[1]] != "#":
                neighbors.append((neighbor, move))
        return neighbors
        

    start_node = Node(start, path_cost=0)
    frontier = [(manhatan_distance(start, end), start_node)] # Se crea una lista donde voy a guardar la distancia manhattan entre mi inicio, final y donde inicio en el laberinto.
    heapq.heapify(frontier) #Convierte la lista frontier en una cola de prioridad (heap). Las colas de prioridades organizan los valores de acuerdo al primer valor de la tupla, es decir, en este caso por la distancia manhatan. Ósea que aquellos nodos con una distancia manhatan menor van a ser los primeros explorados. 
    reached = {start: start_node} 
    """
    El reached guarda las posiciones visitadas como claves y los nodos correspondientes como valores. Es decir, el reached almacena el mejor camino encontrado hasta ahora a cada posición
    explorada. 
    """

    while frontier:
        _, node = heapq.heappop(frontier) # Saca el nodo con menor distancia manhattan de la cola de prioridad, y "_" ignora el primer valor, es decir, ignora el valor de la distancia manhatan y solo toma el nodo.
        if node.position == end:   
            return reconstruct_path(node)

        for neighbor, action in get_neighbors(node.position): # Esto devuelve los nodos válidos donde se puede mover
            new_cost = node.path_cost + 1 
            if neighbor not in reached or new_cost < reached[neighbor].path_cost: # Sí nunca hemos visitado ese vecino (not in reached) o si encontramos un costo menor hacia el vecino actualizamos reached
                reached[neighbor] = Node(neighbor, parent=node, path_cost=new_cost, action=action) # Creamos un nuevo nodo, con el vecino como posición inicial, el parent sería el nodo de donde venimos y el costo
                heapq.heappush(frontier, (manhatan_distance(neighbor, end), reached[neighbor]))

    return None  # No se encontró salida

In [44]:
maze = [
    ["#", "#", "#", "#", "#", "#", "#","#"],
    ["#", "S", "#", " ", "#", " ", "E","#"],
    ["#", " ", " ", " ", "#", " ", " ","#"],
    ["#", " ", "#", " ", " ", " ", "#","#"],
    ["#", "#", "#", "#", "#", "#", "#","#"],
    ["#", "#", "#", "#", "#", "#", "#","#"]
]
path = find_exit(maze)
print("Path to exit:", path)

Path to exit: ([(1, 1), (2, 1), (2, 2), (2, 3), (3, 3), (3, 4), (3, 5), (2, 5), (1, 5), (1, 6)], ['down', 'right', 'right', 'down', 'right', 'right', 'up', 'up', 'right'])


**UNA VEZ SOLUCIONADO EL EJERCICIO, RESPONDA:**

#### **1. ¿Cómo cambia el comportamiento del algoritmo si cambiamos la función de costo?**
##### Dado que la heurística utilizada es aquella por la que calculamos la distancia Manhatan en cada vecino válido que tenemos y lo agregamos a la frontera (cola de prioridad), si bien la distancia Manhatan siempre toda el mejor de los casos, es decir, un camino sin obstáculos es gracias a esta que podemos priorizar los nodos más cercanos hacía el objetivo, y por ende los primeros que vamos a explorar. Así, en caso tal de que se cambiase la lógica de la función de costo, es decir, dejáramos de almacenar la distancia manhatan, entonces ahora pasaríamos a evaluar todas las fronteras posibles y no necesariamente la más cercana de primera (ya no se le da prioridad), lo que podría hacer al algoritmo ineficiente, pues estaría tomando más pasos para hallar la ruta. 
#### **2. ¿Qué sucede si hay múltiples salidas en el laberinto? ¿Cómo podrías modificar el algoritmo para manejar esta situación?**
##### Entendiendo que una salida es un espacio representado por un slash "/" en el laberito y no una pared ("#") que es un obstáculo, la única modificación pertinente sería agregarle un "and" a la condición que verifica si hay una pared ("#"). Esta: if maze[neighbor[0]][neighbor[1]] != "#" and maze[neighbor[0]][neighbor[1]] != "/": , así no tomaría las salidas como vecinos válidos y seguirá su ruta hacia el objetivo.
#### **3. Modifica el laberinto por uno más grande y con otros tipos de obstáculos, además de paredes. ¿Qué limitaciones encuentras en el algoritmo?**

Nota: Resuelve este problema en una celda aparte para mantener la integridad de tu código original.

In [45]:
def find_exit2(maze):
    start = (1, 1)  # Posición inicial basado en la documentación suministrada
    end = (4, 9)    # Posición de la salida basado en la documentación suministrada

    #DEFINA el conjunto de actions posibles#
    actions = {
        "up":(-1,0),
        "down":(1, 0),
        "right":(0, 1),
        "left":(0, -1)
        
    }

    problem=Problem(start, end, actions, maze)#COMPLETE LA DEFINICIÓN DEL OBJETO Y ADAPTELO EN LOS PUNTOS QUE LO REQUIERAN

    def manhatan_distance(pos, goal):
        return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])  # Distancia de Manhattan
    """
    La distancia Manhattan refleja el número mínimo de movimientos requeridos en una cuadrícula donde solo puedes moverte hacia arriba, abajo, derecha o izquierda.
    Ejemplo práctico: Lo que se quiere conseguir con este maze problem
    Sí tengo un robot en la posición (1,1) y quiero llegar a la posición (1,6), la distancia es: 
    abs(1-1) + abs(6-1) = 5. Así, sé que en el mejor de los casos el robot necesitará al menos 5 pasos para llegar al destino. 
    """

    def get_neighbors(pos):  #ESTA ES LA FUNCIÓN QUE DEBERIA AJUSTAR PARA HACER TRACKING DE LOS MOVIMIENTOS (Up, Down, Right, Left)
        neighbors = [] #lista de vecinos
        for move in problem.actions.keys():
            desplazamiento = problem.actions[move]
            neighbor = (pos[0] + desplazamiento[0], pos[1] + desplazamiento[1])

            if maze[neighbor[0]][neighbor[1]] != "#" and maze[neighbor[0]][neighbor[1]] != "*" and maze[neighbor[0]][neighbor[1]] != "/":
                neighbors.append((neighbor, move))
        return neighbors
        

    start_node = Node(start, path_cost=0)
    frontier = [(manhatan_distance(start, end), start_node)] # Se crea una lista donde voy a guardar la distancia manhattan entre mi inicio, final y donde inicio en el laberinto.
    heapq.heapify(frontier) #Convierte la lista frontier en una cola de prioridad (heap). Las colas de prioridades organizan los valores de acuerdo al primer valor de la tupla, es decir, en este caso por la distancia manhatan. Ósea que aquellos nodos con una distancia manhatan menor van a ser los primeros explorados. 
    reached = {start: start_node} 
    """
    El reached guarda las posiciones visitadas como claves y los nodos correspondientes como valores. Es decir, el reached almacena el mejor camino encontrado hasta ahora a cada posición
    explorada. 
    """

    while frontier:
        _, node = heapq.heappop(frontier) # Saca el nodo con menor distancia manhattan de la cola de prioridad, y "_" ignora el primer valor, es decir, ignora el valor de la distancia manhatan y solo toma el nodo.
        if node.position == end:   
            return reconstruct_path(node)

        for neighbor, action in get_neighbors(node.position): # Esto devuelve los nodos válidos donde se puede mover
            new_cost = node.path_cost + 1 
            if neighbor not in reached or new_cost < reached[neighbor].path_cost: # Sí nunca hemos visitado ese vecino (not in reached) o si encontramos un costo menor hacia el vecino actualizamos reached
                reached[neighbor] = Node(neighbor, parent=node, path_cost=new_cost, action=action) # Creamos un nuevo nodo, con el vecino como posición inicial, el parent sería el nodo de donde venimos y el costo
                heapq.heappush(frontier, (manhatan_distance(neighbor, end), reached[neighbor]))

    return None  # No se encontró salida

In [46]:
maze = [
    ["/", "#", "#", "#", "#", "/", "#", "*", "#", "#", "#", "/"],
    ["#", "S", "#", " ", "#", " ", " ", " ", " ", "*", "#", "#"],
    ["#", " ", " ", " ", "#", " ", "#", " ", "/", "#", "#", "#"],
    ["#", " ", "*", " ", " ", " ", "*", " ", "#", "#", "#", "#"],
    ["*", "#", "#", "#", "*", "#", "#", " ", " ", "E", "#", "#"],
    ["#", "#", "#", "#", "#", "/", "#", " ", "#", "#", "#", "#"],
    ["#", "#", "*", "/", "#", "#", "#", " ", "#", "#", "*", "#"],
    ["*", " ", " ", " ", "#", "*", "*", " ", "#", "#", "#", "#"],
    ["#", " ", "#", " ", " ", " ", " ", " ", "#", "#", "#", "#"],
    ["/", " ", "#", "#", "#", "#", "#", "#", "#", "#", "#", "/"]
]
path = find_exit2(maze)
print("Path to exit:", path)

Path to exit: ([(1, 1), (2, 1), (2, 2), (2, 3), (3, 3), (3, 4), (3, 5), (2, 5), (1, 5), (1, 6), (1, 7), (2, 7), (3, 7), (4, 7), (4, 8), (4, 9)], ['down', 'right', 'right', 'down', 'right', 'right', 'up', 'up', 'right', 'right', 'down', 'down', 'down', 'right', 'right'])
